In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for better visualization
sns.set_style("whitegrid")

print("Libraries imported successfully.")

Libraries imported successfully.


load the benin data from data folder

In [4]:

file_path = file_path = '../data/benin-malanville.csv'
df = pd.read_csv(file_path)

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')

print(f"Data loaded successfully with {len(df)} rows and {len(df.columns)} columns.")

Data loaded successfully with 525600 rows and 18 columns.


Summary Statistics and missing value report

In [5]:
print("\n--- 1. Summary Statistics ---")
print(df.describe().T)


--- 1. Summary Statistics ---
                  count        mean         std    min    25%    50%    75%  \
GHI            525600.0  240.559452  331.131327  -12.9   -2.0    1.8  483.4   
DNI            525600.0  167.187516  261.710501   -7.8   -0.5   -0.1  314.2   
DHI            525600.0  115.358961  158.691074  -12.6   -2.1    1.6  216.3   
ModA           525600.0  236.589496  326.894859    0.0    0.0    4.5  463.7   
ModB           525600.0  228.883576  316.536515    0.0    0.0    4.3  447.9   
Tamb           525600.0   28.179683    5.924297   11.0   24.2   28.0   32.3   
RH             525600.0   54.487969   28.073069    2.1   28.8   55.1   80.1   
WS             525600.0    2.121113    1.603466    0.0    1.0    1.9    3.1   
WSgust         525600.0    2.809195    2.029120    0.0    1.3    2.6    4.1   
WSstdev        525600.0    0.473390    0.273395    0.0    0.4    0.5    0.6   
WD             525600.0  153.435172  102.332842    0.0   59.0  181.0  235.1   
WDstdev        525600

In [6]:
print("\n--- 2. Missing Value Report ---")
missing_data = df.isnull().sum()
total_rows = len(df)

# Calculate percentage of nulls
missing_percent = (missing_data / total_rows) * 100
missing_df = pd.DataFrame({'Total Missing': missing_data, 'Percentage': missing_percent})

print(missing_df.sort_values(by='Percentage', ascending=False))

# List columns with >5% nulls
high_nulls = missing_df[missing_df['Percentage'] > 5]
if not high_nulls.empty:
    print("\nColumns with >5% nulls:")
    print(high_nulls)
else:
    print("\nNo columns found with >5% null values.")


--- 2. Missing Value Report ---
               Total Missing  Percentage
Comments              525600       100.0
GHI                        0         0.0
DNI                        0         0.0
DHI                        0         0.0
ModB                       0         0.0
ModA                       0         0.0
RH                         0         0.0
WS                         0         0.0
WSgust                     0         0.0
Tamb                       0         0.0
WSstdev                    0         0.0
WD                         0         0.0
BP                         0         0.0
WDstdev                    0         0.0
Cleaning                   0         0.0
Precipitation              0         0.0
TModA                      0         0.0
TModB                      0         0.0

Columns with >5% nulls:
          Total Missing  Percentage
Comments         525600       100.0
